In [ ]:
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pickle

import sys;       sys.path.append('../util/shared/python/')
from read_GEOSldas          import read_tilecoord, read_obs_param

from mapper_functions import plot_aus_tight_pcm

In [ ]:
species_groups = {
    "SMOS": [0, 1, 2, 3],
    "SMAP": [4, 5, 6, 7],
    "ASCAT": [8, 9, 10],
    "CYGNSS": [11]
}

In [ ]:
# Read in the OL data files

stats_file_OL = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/OLv8_M36_Aus/' \
'OLv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/tmp_stats_OL_obsfrom_DA_20180801_20230801.nc4'

print('reading stats nc4 file '+stats_file_OL)
stats_OL = {}
with Dataset(stats_file_OL,'r') as nc:
    for key, value in nc.variables.items():
        stats_OL[key] = value[:].filled(np.nan)

ts_stats_file_OL = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/OLv8_M36_Aus/' \
'OLv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/stats_OL_DA_Obs_from_DA_OLv8_M36_Aus_201808_202308.pkl'

with open(ts_stats_file_OL, 'rb') as f:
    loaded_data = pickle.load(f)
stats_dict_OL = loaded_data['stats_dict']
date_vec_OL = loaded_data['date_vec']    
date_vec = date_vec_OL    

In [ ]:
# Read in the DA data files
stats_file_DA = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/DAv8_M36_Aus_v3/' \
'DAv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/tmp_stats_DA_20180801_20230801.nc4'
print('reading stats nc4 file '+stats_file_DA)
stats_DA = {}
with Dataset(stats_file_DA,'r') as nc:
    for key, value in nc.variables.items():
        stats_DA[key] = value[:].filled(np.nan)

ts_stats_file_DA = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/DAv8_M36_Aus_v3/' \
'DAv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/stats_DAv8_M36_Aus_201808_202308.pkl'

with open(ts_stats_file_DA, 'rb') as f:
    loaded_data = pickle.load(f)
stats_dict_DA = loaded_data['stats_dict']
date_vec_DA = loaded_data['date_vec']

In [ ]:
# Read in the OO data files (no bias correction for CYGNSS)

stats_file_OO = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/OLv8_M36_Aus/' \
'OLv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/tmp_stats_OL_20180801_20230801.nc4'

print('reading stats nc4 file '+stats_file_OO)
stats_OO = {}
with Dataset(stats_file_OO,'r') as nc:
    for key, value in nc.variables.items():
        stats_OO[key] = value[:].filled(np.nan)

ts_stats_file_OO = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/OLv8_M36_Aus/' \
'OLv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/stats_OLv8_M36_Aus_201808_202308.pkl'

with open(ts_stats_file_OO, 'rb') as f:
    loaded_data = pickle.load(f)
stats_dict_OO = loaded_data['stats_dict']
date_vec_OO = loaded_data['date_vec']      

In [ ]:
# Sample of final compuation of selected diagnostic metrics for OL
 
Nmin = 20

# Then computer metrics of O-F, O-A, etc. based on above computed
N_data = stats_OL['N_data']
O_mean = stats_OL['obs_mean']
# mean(x-y) = E[x] - E[y]   
OmF_mean = stats_OL['obs_mean'] - stats_OL['fcst_mean']
OmA_mean = stats_OL['obs_mean'] - stats_OL['ana_mean']
# var(x-y) = var(x) + var(y) - 2cov(x,y)
# cov(x,y) = E[xy] - E[x]E[y]
OmF_stdv  = np.sqrt(stats_OL['obs_variance'] + stats_OL['fcst_variance'] - \
                    2 * (stats_OL['oxf_mean'] - stats_OL['obs_mean']*stats_OL['fcst_mean']))
                    
OmA_stdv  = np.sqrt(stats_OL['obs_variance'] + stats_OL['ana_variance'] - \
                    2 * (stats_OL['oxa_mean'] - stats_OL['obs_mean']*stats_OL['ana_mean']))

 # "fcstvar" is assumed constant here for convenience. Modify if necessary
OmF_norm_mean = OmF_mean / np.sqrt(stats_OL['obsvar_mean'] + stats_OL['fcstvar_mean']) 
OmF_norm_stdv = np.sqrt(OmF_stdv**2 / (stats_OL['obsvar_mean'] + stats_OL['fcstvar_mean']) )
  
# Mask out data points with insufficent observations using the Nmin threshold
# Do NOT apply to N_data
OmF_mean[     N_data < Nmin] = np.nan
OmF_stdv[     N_data < Nmin] = np.nan
OmF_norm_mean[N_data < Nmin] = np.nan
OmF_norm_stdv[N_data < Nmin] = np.nan
OmA_mean[     N_data < Nmin] = np.nan
OmA_stdv[     N_data < Nmin] = np.nan
N_data[       N_data < Nmin] = 0

OmF_mean_OL = OmF_mean
OmF_stdv_OL = OmF_stdv
OmF_norm_mean_OL = OmF_norm_mean
OmF_norm_stdv_OL = OmF_norm_stdv
OmA_mean_OL = OmA_mean
OmA_stdv_OL = OmA_stdv
N_data_OL = N_data

group_metrics_OL = {}

for group, species_indices in species_groups.items():
    group_metrics_OL[group] = {}
    group_N_data = np.nansum(N_data[:, species_indices], axis=1)
    
    group_metrics_OL[group]['OmF_mean'] = np.nansum(OmF_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['OmF_stdv'] = np.nansum(OmF_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['OmF_norm_mean'] = np.nansum(OmF_norm_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['OmF_norm_stdv'] = np.nansum(OmF_norm_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['OmA_mean'] = np.nansum(OmA_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['OmA_stdv'] = np.nansum(OmA_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OL[group]['Nobs_data'] = group_N_data


In [ ]:

# Sample of final compuation of selected diagnostic metrics for DA

# Then computer metrics of O-F, O-A, etc. based on above computed
N_data = stats_DA['N_data']
O_mean = stats_DA['obs_mean']
# mean(x-y) = E[x] - E[y]
OmF_mean = stats_DA['obs_mean'] - stats_DA['fcst_mean']
OmA_mean = stats_DA['obs_mean'] - stats_DA['ana_mean']
# var(x-y) = var(x) + var(y) - 2cov(x,y)
# cov(x,y) = E[xy] - E[x]E[y]
OmF_stdv  = np.sqrt(stats_DA['obs_variance'] + stats_DA['fcst_variance'] - \
                    2 * (stats_DA['oxf_mean'] - stats_DA['obs_mean']*stats_DA['fcst_mean']))
OmA_stdv  = np.sqrt(stats_DA['obs_variance'] + stats_DA['ana_variance'] - \
                    2 * (stats_DA['oxa_mean'] - stats_DA['obs_mean']*stats_DA['ana_mean']))
# "fcstvar" is assumed constant here for convenience. Modify if necessary
OmF_norm_mean = OmF_mean / np.sqrt(stats_DA['obsvar_mean'] + stats_DA['fcstvar_mean'])
OmF_norm_stdv = np.sqrt(OmF_stdv**2 / (stats_DA['obsvar_mean'] + stats_DA['fcstvar_mean']) )
# Mask out data points with insufficent observations using the Nmin threshold
# Do NOT apply to N_data
OmF_mean[     N_data < Nmin] = np.nan
OmF_stdv[     N_data < Nmin] = np.nan
OmF_norm_mean[N_data < Nmin] = np.nan
OmF_norm_stdv[N_data < Nmin] = np.nan
OmA_mean[     N_data < Nmin] = np.nan
OmA_stdv[     N_data < Nmin] = np.nan
N_data[       N_data < Nmin] = 0
OmF_mean_DA = OmF_mean
OmF_stdv_DA = OmF_stdv
OmF_norm_mean_DA = OmF_norm_mean
OmF_norm_stdv_DA = OmF_norm_stdv
OmA_mean_DA = OmA_mean
OmA_stdv_DA = OmA_stdv
N_data_DA = N_data

group_metrics_DA = {}

for group, species_indices in species_groups.items():
    group_metrics_DA[group] = {}
    group_N_data = np.nansum(N_data[:, species_indices], axis=1)
    
    group_metrics_DA[group]['OmF_mean'] = np.nansum(OmF_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['OmF_stdv'] = np.nansum(OmF_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['OmF_norm_mean'] = np.nansum(OmF_norm_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['OmF_norm_stdv'] = np.nansum(OmF_norm_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['OmA_mean'] = np.nansum(OmA_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['OmA_stdv'] = np.nansum(OmA_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_DA[group]['Nobs_data'] = group_N_data


In [ ]:
# Sample of final compuation of selected diagnostic metrics for OO (no bias correction for CYGNSS)

# Then computer metrics of O-F, O-A, etc. based on above computed
N_data = stats_OO['N_data']
O_mean = stats_OO['obs_mean']
# mean(x-y) = E[x] - E[y]
OmF_mean = stats_OO['obs_mean'] - stats_OO['fcst_mean']
OmA_mean = stats_OO['obs_mean'] - stats_OO['ana_mean']
# var(x-y) = var(x) + var(y) - 2cov(x,y)
# cov(x,y) = E[xy] - E[x]E[y]
OmF_stdv  = np.sqrt(stats_OO['obs_variance'] + stats_OO['fcst_variance'] - \
                    2 * (stats_OO['oxf_mean'] - stats_OO['obs_mean']*stats_OO['fcst_mean']))
OmA_stdv  = np.sqrt(stats_OO['obs_variance'] + stats_OO['ana_variance'] - \
                    2 * (stats_OO['oxa_mean'] - stats_OO['obs_mean']*stats_OO['ana_mean']))
# "fcstvar" is assumed constant here for convenience. Modify if necessary
OmF_norm_mean = OmF_mean / np.sqrt(stats_OO['obsvar_mean'] + stats_OO['fcstvar_mean'])
OmF_norm_stdv = np.sqrt(OmF_stdv**2 / (stats_OO['obsvar_mean'] + stats_OO['fcstvar_mean']) )
# Mask out data points with insufficent observations using the Nmin threshold
# Do NOT apply to N_data
OmF_mean[     N_data < Nmin] = np.nan
OmF_stdv[     N_data < Nmin] = np.nan
OmF_norm_mean[N_data < Nmin] = np.nan
OmF_norm_stdv[N_data < Nmin] = np.nan
OmA_mean[     N_data < Nmin] = np.nan
OmA_stdv[     N_data < Nmin] = np.nan
N_data[       N_data < Nmin] = 0
OmF_mean_OO = OmF_mean
OmF_stdv_OO = OmF_stdv
OmF_norm_mean_OO = OmF_norm_mean
OmF_norm_stdv_OO = OmF_norm_stdv
OmA_mean_OO = OmA_mean
OmA_stdv_OO = OmA_stdv
N_data_OO = N_data

group_metrics_OO = {}

for group, species_indices in species_groups.items():
    group_metrics_OO[group] = {}
    group_N_data = np.nansum(N_data[:, species_indices], axis=1)
    
    group_metrics_OO[group]['OmF_mean'] = np.nansum(OmF_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['OmF_stdv'] = np.nansum(OmF_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['OmF_norm_mean'] = np.nansum(OmF_norm_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['OmF_norm_stdv'] = np.nansum(OmF_norm_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['OmA_mean'] = np.nansum(OmA_mean[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['OmA_stdv'] = np.nansum(OmA_stdv[:, species_indices] * N_data[:, species_indices], axis=1) / group_N_data
    group_metrics_OO[group]['Nobs_data'] = group_N_data

In [ ]:
ftc = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/OLv8_M36_Aus/' \
      'OLv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/figures/OLv8_M36_Aus.ldas_tilecoord.bin'
tc = read_tilecoord(ftc)
n_tile = tc['N_tile']
lat = tc['com_lat']
lon = tc['com_lon']

map_array = np.empty([n_tile, 3])
map_array.fill(np.nan)
map_array[:, 1] = lon
map_array[:, 2] = lat

In [ ]:
for group in species_groups.keys():

    group_name = group
    map_array[:, 0] = group_metrics_OL[group_name]['Nobs_data']
    # Get statistics
    maxval = np.nanmax(map_array[:, 0])
    minval = np.nanmin(map_array[:, 0])
    # Plot group map
    plot_aus_tight_pcm(
        map_array,
        False,
        True,
        f'Ndata {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})',
        'count',
        0,
        3500
    )


group_name = 'SMOS'

map_array[:, 0] = group_metrics_DA[group_name]['Nobs_data']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'Ndata {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    0, 
    1500
)

group_name = 'SMAP'

map_array[:, 0] = group_metrics_DA[group_name]['Nobs_data']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'Ndata {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    0, 
    3500
)

group_name = 'ASCAT'

map_array[:, 0] = group_metrics_DA[group_name]['Nobs_data']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'Ndata {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    0, 
    4000
)

group_name = 'CYGNSS'

map_array[:, 0] = group_metrics_DA[group_name]['Nobs_data']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
total_obs = np.nansum(map_array[:, 0])
print(f'Total observations for {group_name}: {total_obs}')

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'Ndata {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    0, 
    2500
)

denominator = group_metrics_DA['SMAP']['Nobs_data'] / 2
map_array[:, 0] = ((group_metrics_DA['SMAP']['Nobs_data'] - group_metrics_DA['CYGNSS']['Nobs_data']) / denominator) *100

map_array[:, 0] = ((group_metrics_DA['CYGNSS']['Nobs_data'] - (group_metrics_DA['SMAP']['Nobs_data'] / 2)) / denominator) * 100


# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'(CYGNSS - SMAP) / SMAP Ndata \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    '%', 
    -100, 
    100
)

In [ ]:
group_name = 'CYGNSS'

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OL[group_name]['OmF_mean']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL scaled OmF_mean {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    -0.1, 
    0.1
)

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL scaled OmF_mean {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    -0.01, 
    0.01
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OO[group_name]['OmF_mean']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL not scaled OmF_mean {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    -0.1, 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OO[group_name]['OmF_mean'] - group_metrics_OL[group_name]['OmF_mean']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'Difference {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    -0.1, 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_mean']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA scaled OmF_mean {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'count', 
    -0.01, 
    0.01
)

In [ ]:
def convert_stats_dict_to_arrays(stats_dict):
    """Convert dictionary of lists to numpy arrays"""
    array_dict = {}
    
    for key in stats_dict.keys():
        # Convert list to array and reshape
        array_dict[key] = np.array(stats_dict[key])
        
        # Check if we need to handle missing values (-- in data)
        if isinstance(array_dict[key][0], (list, np.ndarray)):
            # Replace '--' with np.nan
            temp_array = []
            for row in array_dict[key]:
                cleaned_row = [np.nan if x == '--' else float(x) for x in row]
                temp_array.append(cleaned_row)
            array_dict[key] = np.array(temp_array)
    
    return array_dict

# Convert dictionary
stats_dict_DA_arrays = convert_stats_dict_to_arrays(stats_dict_DA)
stats_dict_OL_arrays = convert_stats_dict_to_arrays(stats_dict_OL)
stats_dict_OO_arrays = convert_stats_dict_to_arrays(stats_dict_OO)
# Convert date vector to datetime objects
date_vec_DA = [datetime.strptime(date, '%Y%m') for date in date_vec_DA]
date_vec_OL = [datetime.strptime(date, '%Y%m') for date in date_vec_OL]
date_vec_OO = [datetime.strptime(date, '%Y%m') for date in date_vec_OO]

# Print first few dates to verify
print("Sample dates:", date_vec_DA[:3])

In [ ]:
def calculate_weighted_group_stats(stats_dict, species_groups):
    """Calculate weighted statistics for each group"""
    
    n_times = len(stats_dict['OmF_mean'])
    stats = ['OmF_mean', 'OmF_stdv', 'OmA_mean', 'OmA_stdv']
    
    # Initialize output dictionary
    group_stats = {}
    for group in species_groups.keys():
        group_stats[group] = {stat: np.zeros(n_times) for stat in stats}
        group_stats[group]['Ndata'] = np.zeros(n_times)
    
    # Calculate weighted stats for each timestep
    for t in range(n_times):
        for group, indices in species_groups.items():
            # Get weights for this group/time
            weights = stats_dict['Ndata'][t, indices]
            total_weight = np.sum(weights)
            
            if total_weight > 0:
                # Calculate weighted statistics
                for stat in stats:
                    values = stats_dict[stat][t, indices]
                    group_stats[group][stat][t] = np.average(values, weights=weights)
                group_stats[group]['Ndata'][t] = total_weight
            else:
                # Set to NaN if no observations
                for stat in stats:
                    group_stats[group][stat][t] = np.nan
                    
    return group_stats

# Calculate group means
group_ts_DA = calculate_weighted_group_stats(stats_dict_DA_arrays, species_groups)
group_ts_OL = calculate_weighted_group_stats(stats_dict_OL_arrays, species_groups)
group_ts_OO = calculate_weighted_group_stats(stats_dict_OO_arrays, species_groups)

In [ ]:
print("length of date_vec_DA", len(date_vec_DA))
print("length of date_vec", len(date_vec))

In [ ]:
# Create individual plots for each species
for group in species_groups.keys():
    plt.figure(figsize=(10, 6))
    
    # Plot OmF_mean for OL and DA
    mean_OL = np.nanmean(group_ts_OL[group]['OmF_mean'])
    mean_DA = np.nanmean(group_ts_DA[group]['OmF_mean'])
    
    plt.plot(date_vec_DA, group_ts_OL[group]['OmF_mean'], '--', label=f'{group} OL (Mean: {mean_OL:.3f})')
    plt.plot(date_vec_DA, group_ts_DA[group]['OmF_mean'], '-', label=f'{group} DA (Mean: {mean_DA:.3f})')
    
    # Add black dotted line for Y = 0
    plt.axhline(y=0, color='black', linestyle=':', linewidth=1)
    
    # Customize plot
    plt.title(f'O-F Mean for {group}')
    plt.xlabel('Date')
    plt.ylabel('O-F Mean')
    plt.legend()
    
    # Set x-ticks to every 6 months
    # Set x-ticks using datetime array
    plt.xticks(date_vec_DA[::6], rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot OmF_mean for OL and DA
mean_OL = np.nanmean(group_ts_OL['CYGNSS']['OmF_mean'])
mean_DA = np.nanmean(group_ts_DA['CYGNSS']['OmF_mean'])
mean_OO = np.nanmean(group_ts_OO['CYGNSS']['OmF_mean'])

plt.plot(date_vec_DA, group_ts_OL['CYGNSS']['OmF_mean'], '--', label=f'CYGNSS OL (Mean: {mean_OL:.3f})')
plt.plot(date_vec_DA, group_ts_DA['CYGNSS']['OmF_mean'], '-', label=f'CYGNSS DA (Mean: {mean_DA:.3f})')
plt.plot(date_vec_DA, group_ts_OO['CYGNSS']['OmF_mean'], '-', label=f'CYGNSS OL NS (Mean: {mean_OO:.3f})')

# Add black dotted line for Y = 0
plt.axhline(y=0, color='black', linestyle=':', linewidth=1)

# Customize plot
plt.title(f'O-F Mean for {group}')
plt.xlabel('Date')
plt.ylabel('O-F Mean')
plt.legend()

# Set x-ticks to every 6 months
# Set x-ticks using datetime array
plt.xticks(date_vec_DA[::6], rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create individual plots for each species
for group in species_groups.keys():
    plt.figure(figsize=(10, 6))
    
    # Plot OmF_mean for OL and DA
    mean_OL = np.nanmean(group_ts_OL[group]['OmF_stdv'])
    mean_DA = np.nanmean(group_ts_DA[group]['OmF_stdv'])
    
    plt.plot(date_vec_DA, group_ts_OL[group]['OmF_stdv'], '--', label=f'{group} OL (Mean: {mean_OL:.3f})')
    plt.plot(date_vec_DA, group_ts_DA[group]['OmF_stdv'], '-', label=f'{group} DA (Mean: {mean_DA:.3f})')
    
    # Customize plot
    plt.title(f'O-F StdDev for {group}')
    plt.xlabel('Date')
    plt.ylabel('O-F StdDev')
    plt.legend()
    
    # Set y-axis minimum to zero
    # plt.ylim(bottom=0)
    
    # Set x-ticks using datetime array
    plt.xticks(date_vec_DA[::6], rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
group_name = 'CYGNSS'

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    0., 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    0., 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA - OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    -0.01, 
    0.01
)

# Set observation counts from stored metrics
map_array[:, 0] = ((group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']) / group_metrics_OL[group_name]['OmF_stdv']) * 100

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'(DA - OL) / OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    '%', 
    -30, 
    30
)

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    True, 
    True, 
    'DA - CNTL: Δ Relative StdDev of CYGNSS SM OmF', 
    '%', 
    -30, 
    30
)


In [ ]:
group_name = 'SMAP'

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    0., 
    25
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    0., 
    25
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA - OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    -2, 
    2
)

# Set observation counts from stored metrics
map_array[:, 0] = ((group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']) / group_metrics_OL[group_name]['OmF_stdv']) * 100

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'(DA - OL) / OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    '%', 
    -30, 
    30
)


In [ ]:
group_name = 'SMOS'

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    0., 
    25
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    0., 
    25
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA - OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'K', 
    -2, 
    2
)

# Set observation counts from stored metrics
map_array[:, 0] = ((group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']) / group_metrics_OL[group_name]['OmF_stdv']) * 100

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'(DA - OL) / OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    '%', 
    -30, 
    30
)


In [ ]:
group_name = 'ASCAT'

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    0., 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    0., 
    0.1
)

# Set observation counts from stored metrics
map_array[:, 0] = group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'DA - OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    'm3 m-3', 
    -0.01, 
    0.01
)

# Set observation counts from stored metrics
map_array[:, 0] = ((group_metrics_DA[group_name]['OmF_stdv'] - group_metrics_OL[group_name]['OmF_stdv']) / group_metrics_OL[group_name]['OmF_stdv']) * 100

# Get statistics
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])

# Plot group map
plot_aus_tight_pcm(
    map_array, 
    False, 
    True, 
    f'(DA - OL) / OL OmF_stdv {group_name} \n (Max: {maxval:.3g} Min: {minval:.3g})', 
    '%', 
    -30, 
    30
)
